In [22]:
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')

In [14]:
import tensorflow_datasets as tfds

# loading the imdb review dataset
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)
print(len(imdb['test']))

25000


In [15]:
import numpy as np

train_data, test_data = imdb['train'], imdb['test']

training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []


for s,l in train_data:
  training_sentences.append(s.numpy().decode('utf8'))
  training_labels.append(l.numpy())
  
for s,l in test_data:
  testing_sentences.append(s.numpy().decode('utf8'))
  testing_labels.append(l.numpy())
  
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)


In [16]:
vocab_size = 10000              # max vocabulary size
embedding_dim = 16              # the embedding dimension for each word
max_length = 120                # max_length that each sentence can have
trunc_type='post'               # if sentence is more than max_length, it is truncated at the end.
oov_tok = "<OOV>"               # token for words out of vocabulary


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# tokenizer assigns a value to each unique word in the vocabulary
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
# padding ensures each sentence has same number of words by 
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)


In [17]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

# A sample sentence 
print(decode_review(padded[3]))
print(training_sentences[3])

? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? this is the kind of film for a snowy sunday afternoon when the rest of the world can go ahead with its own business as you <OOV> into a big arm chair and <OOV> for a couple of hours wonderful performances from cher and nicolas cage as always gently row the plot along there are no <OOV> to cross no dangerous waters just a warm and witty <OOV> through new york life at its best a family film in every sense and one that deserves the praise it received
This is the kind of film for a snowy Sunday afternoon when the rest of the world can go ahead with its own business as you descend into a big arm-chair and mellow for a couple of hours. Wonderful performances from Cher and Nicolas Cage (as always) gently row the plot along. There are no rapids to cross, no dangerous waters, just a warm and witty paddle through New York life at its best. A family film in every sense and one that deserves the praise it received.


In [18]:
# first layer is an embedding layer which converts every word into a vector of size 'embedding_dim'
# then adding a LSTM layer to the model
# a fully connected hidden layer followed by a output layer

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 64)                12544     
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 390       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 7         
Total params: 172,941
Trainable params: 172,941
Non-trainable params: 0
_________________________________________________________________


In [19]:
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Epoch 1/10
782/782 [==============================] - 33s 42ms/step - loss: 0.5513 - accuracy: 0.7120 - val_loss: 0.3868 - val_accuracy: 0.8286
Epoch 2/10
782/782 [==============================] - 38s 48ms/step - loss: 0.3336 - accuracy: 0.8658 - val_loss: 0.3520 - val_accuracy: 0.8490
Epoch 3/10
782/782 [==============================] - 38s 49ms/step - loss: 0.2608 - accuracy: 0.9006 - val_loss: 0.3609 - val_accuracy: 0.8457
Epoch 4/10
782/782 [==============================] - 39s 50ms/step - loss: 0.2098 - accuracy: 0.9231 - val_loss: 0.3917 - val_accuracy: 0.8375
Epoch 5/10
782/782 [==============================] - 39s 50ms/step - loss: 0.1770 - accuracy: 0.9367 - val_loss: 0.4437 - val_accuracy: 0.8268
Epoch 6/10
782/782 [==============================] - 38s 49ms/step - loss: 0.1469 - accuracy: 0.9492 - val_loss: 0.4802 - val_accuracy: 0.8265
Epoch 7/10
782/782 [==============================] - 38s 49ms/step - loss: 0.1309 - accuracy: 0.9544 - val_loss: 0.5948 - val_accuracy:

  The model achieved a training accuracy of 97% and testing accuracy of 82%

In [20]:
import io

# storing the words in the 'words.tsv' file and their corresponding embedding values in the 'vectors.tsv' file
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('words.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [23]:
sentence = "I really think this is amazing."

sequence = tokenizer.texts_to_sequences([sentence])
model.predict(sequence)

array([[0.9870386]], dtype=float32)

In [24]:
sentence = "Movie was not up to the mark."

sequence = tokenizer.texts_to_sequences([sentence])
model.predict(sequence)

array([[0.30645788]], dtype=float32)

We can see that the model predicts the positivity of both the sample sentences correctly.